<a href="https://colab.research.google.com/github/michelleli246/D4GProject/blob/main/D4GProjectDataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# load in CSVs
from google.colab import files
data_to_load = files.upload()

# data cleaning file: antcleaningsampledata.csv
# taxonomy irn vlookup file: step5lookups.csv

Saving antcleaningsampledata.csv to antcleaningsampledata.csv
Saving step5lookups.csv to step5lookups.csv


In [ ]:
import io

#load in antcleaningsampledata.csv
initial_file = pd.read_csv(io.BytesIO(data_to_load['antcleaningsampledata.csv']),encoding='latin-1',dtype=str)

#load in IRN lookups csv
irn_lookups = pd.read_csv(io.BytesIO(data_to_load['step5lookups.csv']),encoding='latin-1',dtype = str)

In [ ]:
# check out first rows
initial_file.head()

,Lot Number,Collectors Number,Biotic Region,Country,MajorUnit,MinorUnit,Locality Displacement,MidHabitat,Microhabitat,Collection Year,Collection Month,Collection Day,Collectors,LatD,LatM,LatS,LatNS,LongD,LongM,LongS,LongEW,Elevation meters,Elevation meters higher,Elevation feet,Elevation feet higher,Collection Method,Lot Specimen Association,Lots Notes ex Lots DB,Lots Notes: Passed to Object DB,Depository,Object Number Prefix,USNM Collection Object Number,Object Category,Kingdom,Order,Family,Genus,Genus ex TaxonDB,Species,Species ex TaxonDB,Determined by,Workers,Males,Pupae,Alate_Females,Dealate_Females,Larvae,Object Notes,LatLong Source
0,1,TRS920517-01,NEO,Costa Rica,NaN,Las Alturas (Cotón),Las Alturas Biol Stn,NaN,NaN,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,Beginning of C Trail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,TRS920517-01,NEO,Costa Rica,NaN,Las Alturas (Cotón),Las Alturas Biol Stn,NaN,NaN,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,NaN,Beginning of C Trail,NaN,USNMENT,NaN,NaN,NaN,NaN,NaN,NaN,Sericomyrmex,NaN,amabilis,NaN,0,0,0,0,0,0,NaN,NaN
2,2,TRS920517-02,NEO,Costa Rica,NaN,Las Alturas (Cotón),Las Alturas Biol Stn,NaN,rotten log,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,50m onto C Trail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,TRS920517-02,NEO,Costa Rica,NaN,Las Alturas (Cotón),Las Alturas Biol Stn,NaN,rotten log,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,NaN,50m onto C Trail,USNM,USNMENT,00302600,vial,Animalia,Pseudoscorpion,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN
4,3,TRS920517-03,NEO,Costa Rica,NaN,Las Alturas (Cotón),Las Alturas Biol Stn,NaN,Rotten log,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,Hand,Strays,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

## STEP 2

In [ ]:
#duplicate initial df
delete_null_df = initial_file.copy(deep=True)

### Delete rows where certain columns are empty

In [ ]:
#delete rows where columns in no_null_cols are null
no_null_cols = ['Object Number Prefix']

delete_null_df = delete_null_df.dropna(subset = no_null_cols)

## STEP 3

### Add standard val columns: Blue columns

#### Add values to every row

In [ ]:
#duplicate df from step 2 after unnecessary rows were deleted
add_standard_val_df = delete_null_df.copy(deep=True)

In [ ]:
# Standard val column names and their values
#[Col name, Value]

standard_val_cols = [
  ['CatCatalog' , 'Specimen Inventory'],
  ['CatObjectType' , 'Specimen/lot'],
  ['CatOtherNumbersType_tab(1)' , 'Schultz Lot No.'],
  ['CatOtherNumbersSource_tab(1)' , 'AntLab FileMaker Database'],
  ['CatOtherNumbersType_tab(2)' , 'Collector No.'],
  ['CatOtherNumbersSource_tab(2)' , 'label'],
  ['StaRecordStatus_tab' , 'Reviewed'],
  ['StaRecordDate0' , 'November  2020'],
  ['StaRecordRemarks_tab' , 'Submitted by T. Schultz, SI'],
  ['NotNmnhType_tab(1)' , 'Lot Specimen Association'],
  ['NotNmnhWeb_tab(1)' , 'Yes'],
  ['ZooPrepLocationRef_tab.irn' , '10028267'],
  ['ZooPreparationCount_tab' , '1']
]

In [ ]:
#insert standard cols
for i in standard_val_cols:
  add_standard_val_df.insert(0,i[0],i[1])

#### Add standard value to row ONLY if there is a value in another column

In [ ]:
# conditional standard val column names, their values, and the column they depend on
# [Col name, Value, Dependency]

conditional_standard_val_cols = [
  ['NotNmnhType_tab(2)' , 'Collecting Note', 'Lots Notes: Passed to Object DB'],
  ['NotNmnhWeb_tab(2)' , 'Yes', 'Lots Notes: Passed to Object DB']
]

In [ ]:
# insert standard cols, insert value only if the dependent column is not null

for i in conditional_standard_val_cols:
  add_standard_val_df[i[0]] = add_standard_val_df[i[2]].where(add_standard_val_df[i[2]].isna(), i[1])

In [ ]:
add_standard_val_df.head()

,ZooPreparationCount_tab,ZooPrepLocationRef_tab.irn,NotNmnhWeb_tab(1),NotNmnhType_tab(1),StaRecordRemarks_tab,StaRecordDate0,StaRecordStatus_tab,CatOtherNumbersSource_tab(2),CatOtherNumbersType_tab(2),CatOtherNumbersSource_tab(1),CatOtherNumbersType_tab(1),CatObjectType,CatCatalog,Lot Number,Collectors Number,Biotic Region,Country,MajorUnit,MinorUnit,Locality Displacement,MidHabitat,Microhabitat,Collection Year,Collection Month,Collection Day,Collectors,LatD,LatM,LatS,LatNS,LongD,LongM,LongS,LongEW,Elevation meters,Elevation meters higher,Elevation feet,Elevation feet higher,Collection Method,Lot Specimen Association,Lots Notes ex Lots DB,Lots Notes: Passed to Object DB,Depository,Object Number Prefix,USNM Collection Object Number,Object Category,Kingdom,Order,Family,Genus,Genus ex TaxonDB,Species,Species ex TaxonDB,Determined by,Workers,Males,Pupae,Alate_Females,Dealate_Females,Larvae,Object Notes,LatLong Source,NotNmnhType_tab(2),NotNmnhWeb_tab(2)
1,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,1,TRS920517-01,NEO,Costa Rica,NaN,Las Alturas (Cotón),Las Alturas Biol Stn,NaN,NaN,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,NaN,Beginning of C Trail,NaN,USNMENT,NaN,NaN,NaN,NaN,NaN,NaN,Sericomyrmex,NaN,amabilis,NaN,0,0,0,0,0,0,NaN,NaN,Collecting Note,Yes
3,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,2,TRS920517-02,NEO,Costa Rica,NaN,Las Alturas (Cotón),Las Alturas Biol Stn,NaN,rotten log,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,NaN,50m onto C Trail,USNM,USNMENT,00302600,vial,Animalia,Pseudoscorpion,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,Collecting Note,Yes
5,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,3,TRS920517-03,NEO,Costa Rica,NaN,Las Alturas (Cotón),Las Alturas Biol Stn,NaN,Rotten log,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,Hand,Strays,NaN,NaN,USNM,USNMENT,00302601,vial,Animalia,Coleoptera,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN
7,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,4,TRS920616-01,NEO,Nicaragua,NaN,NaN,"La Fundadora (vic. Matagalpa), Cafetal La Runga",coffee plantation,soil,1992,Jun,16,T.R. Schultz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1150,NaN,NaN,NaN,hand,NaN,NaN,NaN,USNM,USNMENT,00302701,vial,Animalia,Hymenoptera,Formicidae,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN
10,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,8,TRS920616-05,NEO,Nicaragua,NaN,NaN,"La Fundadora (vic. Matagalpa), Cafetal La Laguna",NaN,soil,1992,Jun,16,T.R. Schultz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,810,NaN,NaN,NaN,hand,nest series,NaN,road outside of cafetal,NaN,USNMENT,00000001,NaN,NaN,Hymenoptera,Formicidae,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,Collecting Note,Yes


### Rename headers: Green columns, Pt 1 of Yellow columns

In [ ]:
#duplicate standard val df
rename_headers_df = add_standard_val_df.copy(deep=True)

In [ ]:
#cols that will be renamed
#[New col name, Old col name]

rename_cols = [
  ['CatOtherNumbersValue_tab(1)' , 'Lot Number'],
  ['CatOtherNumbersValue_tab(2)' , 'Collectors Number'],
  ['BioEventSiteRef.LocCountry' , 'Country'],
  ['BioEventSiteRef.LocPreciseLocation' , 'Locality Displacement'],
  ['BioEventSiteRef.AquHabitatDescription' , 'MidHabitat'],
  ['BioMicrohabitatDescription' , 'Microhabitat'],
  ['BioEventSiteRef.TerElevationFromMet' , 'Elevation meters'],
  ['BioEventSiteRef.ColCollectionMethod' , 'Collection Method'],
  ['NotNmnhText0(1)' , 'Lot Specimen Association'],
  ['NotNmnhText0(2)' , 'Lots Notes: Passed to Object DB'],
  ['ZooPreparation_tab' , 'Object Category'],

  ['BioEventSiteRef.LocBiogeographicalRegion' , 'Biotic Region'],
  ['BioEventSiteRef.LocProvinceStateTerritory' , 'MajorUnit'],
  ['BioEventSiteRef.LocDistrictCountyShire' , 'MinorUnit'],
  ['BioEventSiteRef.TerElevationToMet' , 'Elevation meters higher']
]

In [ ]:
#rename the headers
for i in rename_cols:
  rename_headers_df = rename_headers_df.rename(columns = {i[1]:i[0]})

### Duplicate column and add new header: Orange columns

In [ ]:
#duplicate standard val df
new_duplicated_df = rename_headers_df.copy(deep=True)

In [ ]:
#cols that will be duplicated with a new header
#[New col name, Col that it is duplicated from]

new_duplicated_cols = [
  ['BioEventSiteRef.ColSiteVisitNumbers_tab','CatOtherNumbersValue_tab(2)']
]

In [ ]:
#add the new columns
for i in new_duplicated_cols:
  new_duplicated_df[i[0]] = new_duplicated_df[i[1]]

### Remove abbrevations, fill in blanks with [Not Stated]: Pt 2 of Yellow columns

In [ ]:
#duplicate the new duplicated cols df
abb_blanks_df = new_duplicated_df.copy(deep=True)

Abbrevations

In [ ]:
#list of abbrevations
#[Full termevations, Abbrevation]

bio_region_abbrevations = [
  ['Neotropical' , 'NEO'],
  ['Palearctic' , 'PAL'],
  ['Afrotropical' , 'AFR'],
  ['Australasian' , 'AUS'],
  ['Neartic' , 'NEA'],
  ['Oceanian' , 'OCE']
]

In [ ]:
full_term = []
abbrevation = []

for i in bio_region_abbrevations:
  full_term.append(i[0])
  abbrevation.append(i[1])

In [ ]:
#replace abbrevations
abb_blanks_df['BioEventSiteRef.LocBiogeographicalRegion'] = abb_blanks_df['BioEventSiteRef.LocBiogeographicalRegion'].replace(abbrevation,full_term)

Blanks

In [ ]:
replace_blanks_cols = [
  ['BioEventSiteRef.LocProvinceStateTerritory'],
  ['BioEventSiteRef.LocDistrictCountyShire']
]

In [ ]:
for i in replace_blanks_cols:
  abb_blanks_df[i] = abb_blanks_df[i].fillna('[Not Stated]')

## STEP 4

In [ ]:
#duplicate the abbrevations and blanks df
concats_df = abb_blanks_df.copy(deep=True)

In [ ]:
# cols that get concatenated into a new column
# [[List of output column names], [List of columns to concatenate], what to put between concatenated values aka a space or no]

cols_to_concat = [
  [['CatBarcode'],['Object Number Prefix','USNM Collection Object Number'],''],
  [['ColDateVisitedFrom','ColDateVisitedTo'],['Collection Day', 'Collection Month', 'Collection Year'], ' '],
  [['LatLatitude_nesttab'],['LatD','LatM','LatS','LatNS'], ' '],
  [['LatLatitude_nesttab'],['LongD','LongM','LongS','LongEW'], ' ']
]

In [ ]:
#add new columns that are concatenations of the columns
for i in cols_to_concat:
  for j in i[0]:
    concats_df[j] = concats_df.fillna('')[i[1]].apply(lambda x: i[2].join(x.astype(str)), axis=1)

In [ ]:
concats_df.head()

,ZooPreparationCount_tab,ZooPrepLocationRef_tab.irn,NotNmnhWeb_tab(1),NotNmnhType_tab(1),StaRecordRemarks_tab,StaRecordDate0,StaRecordStatus_tab,CatOtherNumbersSource_tab(2),CatOtherNumbersType_tab(2),CatOtherNumbersSource_tab(1),CatOtherNumbersType_tab(1),CatObjectType,CatCatalog,CatOtherNumbersValue_tab(1),CatOtherNumbersValue_tab(2),BioEventSiteRef.LocBiogeographicalRegion,BioEventSiteRef.LocCountry,BioEventSiteRef.LocProvinceStateTerritory,BioEventSiteRef.LocDistrictCountyShire,BioEventSiteRef.LocPreciseLocation,BioEventSiteRef.AquHabitatDescription,BioMicrohabitatDescription,Collection Year,Collection Month,Collection Day,Collectors,LatD,LatM,LatS,LatNS,LongD,LongM,LongS,LongEW,BioEventSiteRef.TerElevationFromMet,BioEventSiteRef.TerElevationToMet,Elevation feet,Elevation feet higher,BioEventSiteRef.ColCollectionMethod,NotNmnhText0(1),Lots Notes ex Lots DB,NotNmnhText0(2),Depository,Object Number Prefix,USNM Collection Object Number,ZooPreparation_tab,Kingdom,Order,Family,Genus,Genus ex TaxonDB,Species,Species ex TaxonDB,Determined by,Workers,Males,Pupae,Alate_Females,Dealate_Females,Larvae,Object Notes,LatLong Source,NotNmnhType_tab(2),NotNmnhWeb_tab(2),BioEventSiteRef.ColSiteVisitNumbers_tab,CatBarcode,ColDateVisitedFrom,ColDateVisitedTo,LatLatitude_nesttab
1,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,1,TRS920517-01,Neotropical,Costa Rica,[Not Stated],Las Alturas (Cotón),Las Alturas Biol Stn,NaN,NaN,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,NaN,Beginning of C Trail,NaN,USNMENT,NaN,NaN,NaN,NaN,NaN,NaN,Sericomyrmex,NaN,amabilis,NaN,0,0,0,0,0,0,NaN,NaN,Collecting Note,Yes,TRS920517-01,USNMENT,17 May 1992,17 May 1992,82 50 W
3,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,2,TRS920517-02,Neotropical,Costa Rica,[Not Stated],Las Alturas (Cotón),Las Alturas Biol Stn,NaN,rotten log,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,NaN,50m onto C Trail,USNM,USNMENT,00302600,vial,Animalia,Pseudoscorpion,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,Collecting Note,Yes,TRS920517-02,USNMENT00302600,17 May 1992,17 May 1992,82 50 W
5,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,3,TRS920517-03,Neotropical,Costa Rica,[Not Stated],Las Alturas (Cotón),Las Alturas Biol Stn,NaN,Rotten log,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,Hand,Strays,NaN,NaN,USNM,USNMENT,00302601,vial,Animalia,Coleoptera,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,TRS920517-03,USNMENT00302601,17 May 1992,17 May 1992,82 50 W
7,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,4,TRS920616-01,Neotropical,Nicaragua,[Not Stated],[Not Stated],"La Fundadora (vic. Matagalpa), Cafetal La Runga",coffee plantation,soil,1992,Jun,16,T.R. Schultz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1150,NaN,NaN,NaN,hand,NaN,NaN,NaN,USNM,USNMENT,00302701,vial,Animalia,Hymenoptera,Formicidae,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,TRS920616-01,USNMENT00302701,16 Jun 1992,16 Jun 1992,
10,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,8,TRS920616-05,Neotropical,Nicaragua,[Not Stated],[Not Stated],"La Fundadora (vic. Matagalpa), Cafetal La Laguna",NaN,soil,1992,Jun,16,T.R. Schultz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,810,NaN,NaN,NaN,hand,nest series,NaN,road outside of cafetal,NaN,USNMENT,00000001,NaN,NaN,Hymenoptera,Formic

## STEP 5

### Step 5 Pt 1-3: Add Cla columns

In [ ]:
#duplicate the abbrevations and blanks df
irn_df = concats_df.copy(deep=True)

In [ ]:
# cols that input into the Cla columns and their "backfill" values
# [Cla column name, [list of columns that go into the Cla column], backfill value]

# EXCEPTIONS: 
# ClaKingdom is a direct map, does not have a backfill value
# ClaPhylum is a newly added column, all values will be the backfilled

cla_inputs = [
  ['ClaKingdom', ['Kingdom'], ''],
  # ['ClaPhylum','', 'Arthropoda'], ASK ABOUT THIS

  ['ClaOrder', ['Order'], 'Unidentified'],
  ['ClaFamily', ['Family'], 'Unidentified'],
  ['ClaGenus', ['Genus', 'Genus ex TaxonDB'], 'Unidentified'],
  ['ClaSpecies', ['Species', 'Species ex TaxonDB'], 'sp.']
]

In [ ]:
irn_df = irn_df.reset_index(drop = True)
# generate Cla values
for cla_set in cla_inputs:
  #add empty Cla column
  irn_df[cla_set[0]] = cla_set[2]

  for index, row in irn_df.iterrows():
    #print('here',index,row[cla_set[1]])
    #default input is the backfill value
    cla_val = cla_set[2]

    #check if there's a non null value in the input columns
    for col in cla_set[1]:
      if pd.isna(row[col]) is False:
        cla_val = row[col]

    irn_df.at[index,cla_set[0]] = cla_val

### Step 5 Part 4: Concat the Cla columns to make a new column for the lookup

In [ ]:
# columns that get concatenated for the Cla
cla_concat = ['ClaOrder','ClaFamily','ClaGenus','ClaSpecies']

In [ ]:
#concat into the new col
irn_df['ClaConcatenated'] = irn_df.fillna('')[cla_concat].apply(lambda x: ' '.join(x.astype(str)), axis=1)

### Step 5 Part 5: Left join the input file with the lookup file to get the IRN numbers

In [ ]:
# what to match on and what to pull
col_to_match_on = 'ClaConcatenated'

import_col_to_match_to = 'for match'
import_col_to_include = 'IRN'

In [ ]:
#there are occasionally multiple IRNs that could match to the concatenated Cla value, get the first match
irn_lookups = irn_lookups.groupby('for match',as_index=False).first()

In [ ]:
#merge
irn_df = irn_df.merge(irn_lookups[list((import_col_to_match_to, import_col_to_include))], how = 'left', left_on = col_to_match_on, right_on = import_col_to_match_to)

#rename merged columns
irn_df = irn_df.rename(columns = {'IRN':'IdeTaxonRef_tab.irn'})

In [ ]:
irn_df.head()

,ZooPreparationCount_tab,ZooPrepLocationRef_tab.irn,NotNmnhWeb_tab(1),NotNmnhType_tab(1),StaRecordRemarks_tab,StaRecordDate0,StaRecordStatus_tab,CatOtherNumbersSource_tab(2),CatOtherNumbersType_tab(2),CatOtherNumbersSource_tab(1),CatOtherNumbersType_tab(1),CatObjectType,CatCatalog,CatOtherNumbersValue_tab(1),CatOtherNumbersValue_tab(2),BioEventSiteRef.LocBiogeographicalRegion,BioEventSiteRef.LocCountry,BioEventSiteRef.LocProvinceStateTerritory,BioEventSiteRef.LocDistrictCountyShire,BioEventSiteRef.LocPreciseLocation,BioEventSiteRef.AquHabitatDescription,BioMicrohabitatDescription,Collection Year,Collection Month,Collection Day,Collectors,LatD,LatM,LatS,LatNS,LongD,LongM,LongS,LongEW,BioEventSiteRef.TerElevationFromMet,BioEventSiteRef.TerElevationToMet,Elevation feet,Elevation feet higher,BioEventSiteRef.ColCollectionMethod,NotNmnhText0(1),Lots Notes ex Lots DB,NotNmnhText0(2),Depository,Object Number Prefix,USNM Collection Object Number,ZooPreparation_tab,Kingdom,Order,Family,Genus,Genus ex TaxonDB,Species,Species ex TaxonDB,Determined by,Workers,Males,Pupae,Alate_Females,Dealate_Females,Larvae,Object Notes,LatLong Source,NotNmnhType_tab(2),NotNmnhWeb_tab(2),BioEventSiteRef.ColSiteVisitNumbers_tab,CatBarcode,ColDateVisitedFrom,ColDateVisitedTo,LatLatitude_nesttab,ClaKingdom,ClaOrder,ClaFamily,ClaGenus,ClaSpecies,ClaConcatenated,for match,IdeTaxonRef_tab.irn
0,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,1,TRS920517-01,Neotropical,Costa Rica,[Not Stated],Las Alturas (Cotón),Las Alturas Biol Stn,NaN,NaN,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,NaN,Beginning of C Trail,NaN,USNMENT,NaN,NaN,NaN,NaN,NaN,NaN,Sericomyrmex,NaN,amabilis,NaN,0,0,0,0,0,0,NaN,NaN,Collecting Note,Yes,TRS920517-01,USNMENT,17 May 1992,17 May 1992,82 50 W,,Unidentified,Unidentified,Sericomyrmex,amabilis,Unidentified Unidentified Sericomyrmex amabilis,NaN,NaN
1,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,2,TRS920517-02,Neotropical,Costa Rica,[Not Stated],Las Alturas (Cotón),Las Alturas Biol Stn,NaN,rotten log,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,hand,strays,NaN,50m onto C Trail,USNM,USNMENT,00302600,vial,Animalia,Pseudoscorpion,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,Collecting Note,Yes,TRS920517-02,USNMENT00302600,17 May 1992,17 May 1992,82 50 W,Animalia,Pseudoscorpion,Unidentified,Unidentified,sp.,Pseudoscorpion Unidentified Unidentified sp.,NaN,NaN
2,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,3,TRS920517-03,Neotropical,Costa Rica,[Not Stated],Las Alturas (Cotón),Las Alturas Biol Stn,NaN,Rotten log,1992,May,17,T.R. Schultz,8,57,NaN,N,82,50,NaN,W,1540,NaN,NaN,NaN,Hand,Strays,NaN,NaN,USNM,USNMENT,00302601,vial,Animalia,Coleoptera,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,TRS920517-03,USNMENT00302601,17 May 1992,17 May 1992,82 50 W,Animalia,Coleoptera,Unidentified,Unidentified,sp.,Coleoptera Unidentified Unidentified sp.,Coleoptera Unidentified Unidentified sp.,10571462
3,1,10028267,Yes,Lot Specimen Association,"Submitted by T. Schultz, SI",November 2020,Reviewed,label,Collector No.,AntLab FileMaker Database,Schultz Lot No.,Specimen/lot,Specimen Inventory,4,TRS920616-01,Neotropical,Nicaragua,[Not Stated],[Not Stated],"La Fundadora (vic. Matagalpa), Cafetal La Runga",coffee plantation,soil,1992,Jun,16,T.R. Schultz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1150,NaN,NaN,NaN,hand,NaN,NaN,NaN,USNM,USNMENT,00302701,vial,Animalia,Hymenoptera,Formicidae,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,TRS920616-01,USNMENT00302701,16 Jun 1992,16 Jun 1992,,Animalia,Hymenoptera,Formicid

## STEP 6

### Step 6 Part 1: Only keep the columns you need

In [ ]:
#check which columns are available:
irn_df.columns

Index(['ZooPreparationCount_tab', 'ZooPrepLocationRef_tab.irn',
       'NotNmnhWeb_tab(1)', 'NotNmnhType_tab(1)', 'StaRecordRemarks_tab',
       'StaRecordDate0', 'StaRecordStatus_tab', 'CatOtherNumbersSource_tab(2)',
       'CatOtherNumbersType_tab(2)', 'CatOtherNumbersSource_tab(1)',
       'CatOtherNumbersType_tab(1)', 'CatObjectType', 'CatCatalog',
       'CatOtherNumbersValue_tab(1)', 'CatOtherNumbersValue_tab(2)',
       'BioEventSiteRef.LocBiogeographicalRegion',
       'BioEventSiteRef.LocCountry',
       'BioEventSiteRef.LocProvinceStateTerritory',
       'BioEventSiteRef.LocDistrictCountyShire',
       'BioEventSiteRef.LocPreciseLocation',
       'BioEventSiteRef.AquHabitatDescription', 'BioMicrohabitatDescription',
       'Collection Year', 'Collection Month', 'Collection Day', 'Collectors',
       'LatD', 'LatM', 'LatS', 'LatNS', 'LongD', 'LongM', 'LongS', 'LongEW',
       'BioEventSiteRef.TerElevationFromMet',
       'BioEventSiteRef.TerElevationToMet', 'Elevation feet',
 

In [ ]:
#out of the available columns, only include the ones needed, in the corrected order

needed_columns = [
  'CatCatalog',
  'CatObjectType',
  'StaRecordStatus_tab',
  'StaRecordDate0',
  'StaRecordRemarks_tab',
  'CatOtherNumbersType_tab(1)',
  'CatOtherNumbersSource_tab(1)',
  'CatOtherNumbersValue_tab(1)',
  'CatOtherNumbersType_tab(2)',
  'CatOtherNumbersSource_tab(2)',
  'CatOtherNumbersValue_tab(2)',
  'BioEventSiteRef.ColSiteVisitNumbers_tab',
  'BioEventSiteRef.LocBiogeographicalRegion',
  'BioEventSiteRef.LocCountry',
  'BioEventSiteRef.LocProvinceStateTerritory',
  'BioEventSiteRef.LocDistrictCountyShire',
  'BioEventSiteRef.LocPreciseLocation',
  'BioEventSiteRef.AquHabitatDescription',
  'BioMicrohabitatDescription',
  'ColDateVisitedFrom',
  'ColDateVisitedTo',
  'Collectors',
  'LatLatitude_nesttab',
  'LatLatitude_nesttab',
  'BioEventSiteRef.TerElevationFromMet',
  'BioEventSiteRef.TerElevationToMet',
  'BioEventSiteRef.ColCollectionMethod',
  'NotNmnhText0(1)',
  'NotNmnhType_tab(1)',
  'NotNmnhWeb_tab(1)',
  'NotNmnhText0(2)',
  'NotNmnhType_tab(2)',
  'NotNmnhWeb_tab(2)',
  'CatBarcode',
  'ZooPreparation_tab',
  'ZooPrepLocationRef_tab.irn',
  'ZooPreparationCount_tab',
  'IdeTaxonRef_tab.irn',
  'ClaKingdom',
  'ClaOrder',
  'ClaFamily',
  'ClaGenus',
  'ClaSpecies',
  'Determined by',
  'Workers',
  'Males',
  'Pupae',
  'Alate_Females',
  'Dealate_Females',
  'Larvae',
  'Object Notes',
  'LatLong Source'

  
]

# Data Export

In [ ]:
from google.colab import files

In [ ]:
irn_df[needed_columns].to_csv('export.csv', index = False)

In [ ]:
files.download('export.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>